## Tabular training

In [7]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib pandas

To illustrate the tabular application, we will use the example of the Adult dataset where we have to predict if a person is earning more or less than $50k per year using some general data.

In [8]:
from fastai.tabular.all import *
from pathlib import Path
import pandas as pd

## Variables

In [9]:
modelName = 'stockScreenerV5.pkl'
trainingDataName = 'stockData.csv'
trainingFolder = Path.cwd().parent / 'TrainingData'
modelFolder = Path.cwd().parent.parent / 'TrainedModels'

# Training parameters
yNames = ['Future Year Change']
catNames = ['Industry']
contNames = [
    'Open',
    'High', 
    'Low', 
    'Close', 
    'Volume', 
    'Dividends', 
    'Stock Splits', 
    'EV/EBIT', 
    'ROIC'
]

We can download a sample of this dataset with the usual untar_data command:

In [10]:
dataPath = Path()
dataPath.ls()

(#3) [Path('app.ipynb'),Path('stockFetcher.ipynb'),Path('tabular.ipynb')]

Then we can have a look at how the data is structured:

In [11]:
df = pd.read_csv(trainingFolder/trainingDataName)
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Future Year Change,Ticker,Industry,EV/EBIT,ROIC
0,2019-01-17 00:00:00-05:00,36.820317,37.646508,36.595861,37.216698,119284800,0.0,0.0,1.075702,AAPL,Consumer Electronics,10.460257,0.075524
1,2019-01-18 00:00:00-05:00,37.608313,37.699051,37.245361,37.445942,135004000,0.0,0.0,1.049015,AAPL,Consumer Electronics,10.519030,0.075102
2,2019-01-22 00:00:00-05:00,37.348042,37.424451,36.443052,36.605427,121576000,0.0,0.0,1.103545,AAPL,Consumer Electronics,10.303540,0.076673
3,2019-01-23 00:00:00-05:00,36.808387,37.044783,36.223369,36.753468,92522400,0.0,0.0,1.105161,AAPL,Consumer Electronics,10.341495,0.076391
4,2019-01-24 00:00:00-05:00,36.798824,36.887172,36.232910,36.462139,101766000,0.0,0.0,1.115866,AAPL,Consumer Electronics,10.266805,0.076947


Some of the columns are continuous (like age) and we will treat them as float numbers we can feed our model directly. Others are categorical (like workclass or education) and we will convert them to a unique index that we will feed to embedding layers. We can specify our categorical and continuous column names, as well as the name of the dependent variable in TabularDataLoaders factory methods:

In [12]:
dls = TabularDataLoaders.from_csv(trainingFolder/trainingDataName, path=dataPath, 
    y_names=yNames,
    cat_names=catNames,
    cont_names=contNames,
    procs = [Categorify, FillMissing, Normalize])

c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

The last part is the list of pre-processors we apply to our data:

* Categorify is going to take every categorical variable and make a map from integer to unique categories, then replace the values by the corresponding index.
* FillMissing will fill the missing values in the continuous variables by the median of existing values (you can choose a specific value if you prefer)
* Normalize will normalize the continuous variables (subtract the mean and divide by the std)

To further expose what’s going on below the surface, let’s rewrite this utilizing fastai’s TabularPandas class. We will need to make one adjustment, which is defining how we want to split our data. By default the factory method above used a random 80/20 split, so we will do the same:

In [13]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df))

In [14]:
to = TabularPandas(df, procs=[Categorify, FillMissing,Normalize],
    y_names=yNames,
    cat_names = catNames,
    cont_names = contNames,
    splits=splits)

c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

Once we build our TabularPandas object, our data is completely preprocessed as seen below:

In [15]:
to.xs.iloc[:2]

,Industry,EV/EBIT_na,ROIC_na,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,ROIC
65219,2,1,1,-0.520946,-0.521226,-0.520961,-0.521582,0.116744,-0.056963,-0.009258,-0.957391,0.008751
60183,32,1,1,-0.296708,-0.297060,-0.295923,-0.294150,-0.165204,-0.056963,-0.009258,1.712893,-0.003221


Now we can build our DataLoaders again:

In [16]:
dls = to.dataloaders(bs=64)

The show_batch method works like for every other application:

In [17]:
dls.show_batch()

,Industry,EV/EBIT_na,ROIC_na,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,ROIC,Future Year Change
0,Internet Content & Information,False,False,114.849998,116.660005,114.809997,115.059998,1.667699e+06,1.257099e-10,-2.988645e-11,-1.718386,-0.459734,0.260125
1,Internet Content & Information,False,False,162.529999,164.960007,156.259995,156.839996,5.463900e+06,1.257099e-10,-2.988645e-11,-1.129568,-0.699383,-0.153341
2,Semiconductor Equipment & Materials,False,False,277.841156,278.969145,276.397702,278.959594,4.100992e+05,1.257099e-10,-2.988645e-11,27.796419,0.028421,0.650050
3,Electronic Gaming & Multimedia,False,False,82.486367,86.406600,81.897851,84.394279,2.525101e+06,1.257099e-10,-2.988645e-11,-3.425471,-0.230625,-0.026012
4,Auto Manufacturers,False,False,257.399993,257.790012,244.479994,244.880002,1.275241e+08,1.257099e-10,-2.988645e-11,52.510860,0.015045,0.038345
5,Banks - Diversified,False,False,87.501946,88.775435,87.262011,88.295570,8.543008e+05,1.257099e-10,-2.988645e-11,-19.177454,-0.041194,0.071970
6,Medical Instruments & Supplies,False,False,193.449997,194.449996,189.080000,191.080000,1.907300e+06,1.257099e-10,-2.988645e-11,54.089061,0.014606,0.534697
7,Semiconductors,False,False,97.971122,99.585082,97.438857,99.464897,3.974300e+06,1.257099e-10,-2.988645e-11,40.971539,0.019282,-0.009520
8,Banks - Diversified,False,False,60.078902,60.519265,59.926118,60.033967,1.239901e+06,1.257099e-10,-2.988645e-11,1.322960,0.597146,-0.001289
9,Auto Manufacturers,False,False,245.000003,258.000000,244.860003,256.489987,1.294696e+08,1.257099e-10,-2.988645e-11,55.068334,0.014346,-0.102616


We can define a model using the tabular_learner method. When we define our model, fastai will try to infer the loss function based on our y_names earlier.

Note: Sometimes with tabular data, your y’s may be encoded (such as 0 and 1). In such a case you should explicitly pass y_block = CategoryBlock in your constructor so fastai won’t presume you are doing regression.

In [18]:
learn = tabular_learner(dls, metrics=[rmse, mae])

And we can train that model with the fit_one_cycle method (the fine_tune method won’t be useful here since we don’t have a pretrained model).

In [19]:
learn.fit_one_cycle(15)

epoch,train_loss,valid_loss,_rmse,mae,time
0,0.427875,4995.292969,70.677383,1.086637,00:13
1,0.594680,263129.750000,512.961914,5.598451,00:12
2,0.449124,379.447845,19.479422,0.553787,00:11
3,0.529573,125575.117188,354.365997,3.954214,00:12
4,0.413462,1.109847,1.053493,0.359428,00:11
5,0.363118,0.722575,0.850044,0.354828,00:11
6,0.413132,3.349041,1.830039,0.376630,00:11
7,0.351818,136242.750000,369.110870,4.176270,00:11
8,0.366212,14891.972656,122.032661,1.608136,00:11
9,0.395917,2.530036,1.590609,0.374007,00:11


We can then have a look at some predictions:

In [20]:
learn.show_results()

,Industry,EV/EBIT_na,ROIC_na,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,ROIC,Future Year Change,Future Year Change_pred
0,26.0,1.0,1.0,-0.413745,-0.415138,-0.412713,-0.414264,-0.240412,-0.056963,-0.009258,-0.497465,-0.002546,0.140651,0.048670
1,21.0,1.0,1.0,-0.428697,-0.428214,-0.427247,-0.426496,-0.181733,-0.056963,-0.009258,-0.495907,-0.002549,0.598612,0.407921
2,32.0,1.0,1.0,0.171487,0.162710,0.173379,0.164953,-0.285751,-0.056963,-0.009258,0.188659,-0.003027,0.097198,0.189551
3,31.0,1.0,1.0,-0.184205,-0.185171,-0.183585,-0.188451,-0.226052,-0.056963,-0.009258,0.698500,-0.003131,0.654475,0.275240
4,14.0,1.0,1.0,-0.345209,-0.332079,-0.350455,-0.340387,0.147184,-0.056963,-0.009258,-0.144277,-0.002891,-0.031260,0.150372
5,31.0,1.0,1.0,-0.131475,-0.135153,-0.133033,-0.138144,-0.263157,-0.056963,-0.009258,0.765186,-0.003140,0.465471,0.280961
6,9.0,1.0,1.0,0.027515,0.019909,0.029662,0.023117,-0.258667,-0.056963,-0.009258,-0.086894,-0.002921,0.205255,0.169231
7,6.0,1.0,1.0,-0.271659,-0.272487,-0.276128,-0.279215,-0.224388,-0.056963,-0.009258,0.249540,-0.003043,3.528537,2.093591
8,3.0,1.0,1.0,-0.392763,-0.391856,-0.390514,-0.389584,-0.285926,-0.056963,-0.009258,-0.561643,-0.002423,-0.162377,-0.029762


## Export the model

In [21]:
learn.export(modelFolder / f'{modelName}')


To get prediction on a new dataframe, you can use the test_dl method of the DataLoaders. That dataframe does not need to have the dependent variable in its column.

In [22]:
from ipynb.fs.full.stockFetcher import getTickerData

In [29]:
predictionTarget = 'AAPL'

test_df = getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
	test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)
test_df.head()

c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
c:\Users\Gamer\miniconda3\Lib\site-packages\fastai\tabular\core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Industry
0,232.115005,232.289993,228.479996,229.979996,63906031,0.0,0.0,59.880474,3.458416e+12,0.013193,Consumer Electronics


In [30]:
prediction = learn.get_preds(dl=dl)
print(f"Prediction for {predictionTarget}:")
print(f"{prediction[0][0][0].item() * 100:.2f}%")

Prediction for AAPL:
-1.11%


Note:
Since machine learning models can’t magically understand categories it was never trained on, the data should reflect this. If there are different missing values in your test data you should address this before training